<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
</div>

In [ ]:
from plan import plan; plan("modules", "modules")

# pour réutiliser du code en Python

* fonctions
  * pas d'état après exécution
* **modules**
  * **garde l'état**
  * **une seule instance par programme**
* classes
  * instances multiples
  * chacune garde l'état
  * héritage

# à quoi sert un module ?

* réutilisation du code
  * un module peut être importé n’importe où
* séparation de l’espace de nommage
  * un module définit essentiellement un espace de nommage
* utilisation des modules
  * un fichier top-level (celui qui est exécuté)  
    importe des modules

  * chaque module peut également importer d’autres modules

# à quoi sert un module ?

* on peut voir les modules comme des boîtes à outils
  * que `import` permet de charger dans son espace de travail
* des centaines de modules sont livrés avec Python
  * c’est la librairie standard 
* des milliers de librairies tierces sont disponibles
  * voir PyPI - the Python Package Index  
    https://pypi.org/

# création d’un module

* un module est un objet python 
  * correspondant au chargement 
  * d'un fichier ou répertoire source
* dans le cas d'un répertoire on parle alors d'un *package*
* le nom d'un fichier doit finir par `.py`
* le préfixe suit les règles des variables
  * i.e. pas de `-` mais des `_`

# importation d’un module

In [ ]:
!cat mod.py

In [ ]:
# je peux l'importer
import mod

In [ ]:
mod

In [ ]:
mod.spam('good')

* le nom `mod` dans `import mod` sert à trouver le fichier `mod.py`, 
* mais **aussi à nommer** l’objet représentant le module importé
* la syntaxe `mod.` donne accès aux **attributs** du module

# définition de la notion d’attribut

* un attribut est une annotation sur un objet (ici le module `mod`)
  * qui associe un nom (ici `spam`) à un autre objet (la fonction)
  * on référence un attribut par `obj.attribute`
* un attribut n'est **pas une variable**
  * les variables sont résolues par liaison lexicale
  * les attributs sont résolus à run-time
  * on en reparlera longuement

### définition de la notion d’attribut

* on peut attacher un attribut à une grande variété d'objets
  * modules, packages, classes, instances, fonctions, ..
* mais pas attacher aux classes ni instances de classes natives

In [ ]:
# on ne peut pas attacher d'attribut aux classes natives 
x = 3
try:
    x.foo = 12
except AttributeError as e:
    print("OOPS", e)

### définition de la notion d’attribut

* les familles d'objets où les attributs sont les plus utilisés
  * modules et packages - on va le voir tout de suite
  * instances et classes - pour la POO
  * fonctions - cf. introspection

### définition de la notion d’attribut

on verra que c'est le coeur de la POO

In [ ]:
# en anticipant un peu
# je crée un classe vide
class Foo: 
    pass
# et une instance de cette classe
foo = Foo()
# je peux créer l'attribut 'name'
foo.name = 'Jean'

# espace de nommage

* un espace de nommage est une association entre attributs et objets
* souvent implémenté par un dictionnaire appelé `__dict__`
* deux espaces de nom sont étanches
  * peuvent avoir tous les deux un attribut disons `name`

### espace de nommage

* un module est un bon exemple d'espace de nommage
* les symboles (fonctions, variables, classes) 
  * définis au top-level dans le module, e.g. globales
  * sont ajoutés dans l'espace de nommage attaché au module
* ex: `mod.spam`
  * correspond à la clé `spam` dans `mod.__dict__`

In [ ]:
mod.spam is mod.__dict__['spam']

### `from module import name`

In [ ]:
from mod import spam
spam('direct') 

In [ ]:
# un peu comme
# spam = mod.spam

* `from mod import spam`
  * copie le nom d’attribut `spam`
  * du module `mod`
  * dans l’espace de nommage local
  * plus besoin de la référence au nom du module

### `import modulename as name`

In [ ]:
import mod as mymod
mymod.spam("module renamed")

In [ ]:
# un peu comme
# import mod
# mymod = mod
# del mod

### `from modulename import name as newname`

In [ ]:
from mod import spam as myspam
myspam('renamed function')

In [ ]:
# un peu comme
# import mod
# myspam = mod.spam
# del mod

### `import dir.dir2.modulename`

In [ ]:
!cat pack1/pack2/mod.py

In [ ]:
import pack1.pack2.mod
pack1.pack2.mod.FOO

* on peut donc importer un sous-module dans un package  
  on reparlera plus longuement des packages

* on peut aussi utiliser `as`:

In [ ]:
import pack1.pack2.mod as submod
submod.eggs()

### autres importations

In [ ]:
from mod import *
spam('star')

In [ ]:
# un peu comme
# mod.spam = spam
# mod.GLOBALE = GLOBALE

* `from mod import *` 
  * copie le nom de **tous** les attributs du module
  * dans l’espace de nommage local
  * plus besoin donc non plus de la référence au nom du module
* remarque: je **déconseille d'éviter** cette directive dans du code de production
  * on perd la traçabilité des symboles importés

# que fait une importation ?

* trouver le fichier correspondant au module 
  * on ne met pas le `.py` du fichier lors d’un import
* compiler (si besoin) le module en byte-code
* charger le module pour construire les objets qu’il définit
  * et les ranger dans les attributs du module
* affecter la variable locale à l'objet module

### byte-code

* en première approximation,   
  vous pouvez ignorer totalement les `.pyc`

* Python se débrouille pour les recompiler au besoin
* les `.pyc` ne sont générés que par les imports,  
  et **pas** pour le point d'entrée

* les `.pyc` sont dans un répertoire `__pycache__`

### localisation du fichier du module

* localisation en parcourant dans l’ordre
  * répertoire où se trouve le point d'entrée 
  * `PYTHONPATH` : variable d’environnement de l’OS
  * répertoires des librairies standards
* `sys.path` contient la liste des répertoires parcourus
  * on peut modifier `sys.path` à l’exécution

### importation d’un module

* comme l’importation est une opération lourde, un module n’est chargé qu’**une seule fois** 
  * les imports suivants réutilisent le module déjà présent en mémoire
* pour importer de nouveau un module (avec une réexécution du code) il faut utiliser la fonction `imp.reload()`
  * utile principalement lors de la mise au point

### importation d’un module

* sous IPython, il existe une extension qui simplifie la vie
  * pour recharger les modules modifiés
  * logique en développement
  * pas utile en production

In [ ]:
%load_ext autoreload
%autoreload 2

### importation d’un module

In [ ]:
!cat toplevel.py

In [ ]:
import toplevel
toplevel.eggs

In [ ]:
toplevel.eggs = 2
import toplevel
toplevel.eggs

# importation d’un module

In [ ]:
# pour recharger un fichier 
import importlib
importlib.reload(toplevel);

In [ ]:
toplevel.eggs

**Note** l'ancien module `imp` est obsolète

# pièges de l’importation

* les instructions `import` et `from` sont des affectations implicites de variables
  * on a donc le problème des références partagées sur des mutables

# pièges de l’importation

In [ ]:
import math
math.pi = 10.

* en fait je viens de modifier `math.pi` **pour tout mon programme !!**
* on n’a pas le problème avec `from` parce que ça crée une copie locale du nom

In [ ]:
from math import pi
pi = 10
# les autres modules ne sont pas impactés

In [ ]:
!cat spam.py

In [ ]:
!cat egg.py

In [ ]:
!python3 egg.py

### exécuter un module comme un script

* un module peut avoir deux rôles
  * un module classique qui doit être importé
  * un script exécutable
* tous les modules ont un nom qui est défini par la variable `__name__`
* le nom d’un module est défini par l’import

In [ ]:
!cat toplevel.py

In [ ]:
import toplevel
print(toplevel.__name__) 

### exécuter un module comme un script

* si le module est le point d'entrée, (`python3 foo.py`)  
  son exécution n’est pas le résultat d’un import

* alors `__name__` est mis à la chaîne  `__main__`
* en faisant un test sur `__name__` dans le module,  
  on peut écrire un code qui ne s’exécute  
  que lorsque le module est le point d'entrée

```python
# voici un idiome fréquent à la fin d'un source Python
if __name__ == '__main__':
    test_module()
```

### exécuter un module comme un script

In [ ]:
!cat samples/fib.py

In [ ]:
# À la ligne de commande on a
!python3 samples/fib.py

In [ ]:
# mais à l'import il ne se passe rien
from samples.fib import fib

* on peut utiliser cette fonctionnalité pour faire des tests unitaires
* mais ce n'est guère utilisé en production (trop limité)

### exécuter un module comme un script

* on peut aussi lancer Python en mode **interactif**

```
$ python3 -i fib.py
1 1 2 3 5 8 13 21 34 
>>>
```

### attributs d’un module

* on accède à tous les attributs d’un module en utilisant
  * `globals()` retourne l’espace de nommage du module courant
  * `locals()` retourne l’espace de nommage à l’endroit de l’appel
  * `vars(module)` retourne l’espace de nommage de module (équivalent à `module.__dict__`)
  * `dir(module)` liste les attributs

### attributs d’un module

In [ ]:
foo = 10
g = globals()
type(g)
list(g.keys())[-10:]

In [ ]:
##
##
'foo' in g

In [ ]:
g['foo']

In [ ]:
# pour les geeks
import sys
(sys.modules[__name__]
  .__dict__['foo'] is foo)

In [ ]:
# si on n'est pas dans une fonction ou une classe,
# locals() et globals() retournent la même chose
locals() == globals()

### attributs d’un module

In [ ]:
# par contre dans une fonction c'est différent
def f():
    tutu = 12
    print(f"tutu dans globals ? : {'tutu' in globals()}")
    print(f"tutu dans locals ? : {'tutu' in locals()}")
    print(f"foo dans globals ? : {'foo' in globals()}")
    print(f"foo dans locals ? : {'foo' in locals()}")
f()

### attributs d’un module

In [ ]:
try:
    print(__dict__)
except NameError as e:
    print("OOPS", e)

* l’attribute `__dict__` est  un attribut spécial pour un module, il n’est pas un nom global
* on ne peut donc pas y accéder sans utiliser un nom qualifié `module.__dict__`.

### attributs d’un module

* `sys.modules` est un dictionnaire de tous les modules chargés
  nom → *objet module*

* `sys.modules[__name__]` 
  permet de retrouver l'objet module  courant

* `sys.modules[__name__].__dict__` 
  est l’espace de nommage du module courant

In [ ]:
sys.modules[__name__].__dict__ == globals()

# notions avancées

* un import importe tous les noms d’un module
* donc un client peut les modifier 
* il existe un convention de nommage
* tous les noms qui commencent par un underscore (`_`) 
  sont privés au module, ne font pas partie de l'API

* ça n’est qu’une convention, mais c’est généralement suffisant

### ordre dans un module

* l’ordre des déclarations dans un module à de l’importance
* le code en dehors des fonctions est exécuté à l’import
* le code dans les fonctions n’est exécuté que lors de l’appel des fonctions

### ordre dans un module

In [ ]:
try:
    func1()    # erreur pas encore déclarée
except:
    import traceback
    traceback.print_exc()

In [ ]:
def func1():
    func2()    # OK, func2() est déclarée après

In [ ]:
try:
    func1()    # erreur func2() pas encore déclarée
except:
    import traceback; traceback.print_exc()

In [ ]:
def func2():
    print("in func2")

In [ ]:
func1()            # OK func1() et func2() sont
                   # déclarées

### notions avancées

* pour importer un module si on a son nom dans une chaîne
  * voir la fonction `importlib.import_module`

In [ ]:
import importlib
nom_module = "math"
math2 = importlib.import_module(nom_module)
math2.e

In [ ]:
# souvenez vous que celui-là, on l'a modifié sauvagement
math2.pi

* `exec` est déconseillé pour ce genre d'usages